In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

### Load data and split

In [2]:
loaded_images = np.load('images.npy')
was_loaded = np.load('was_loaded.npy')

loaded_images = loaded_images[was_loaded == 1]
loaded_images = loaded_images / 255.0

In [3]:
y_train = pd.read_csv('../data/numerical/y_train.csv', sep = ';',index_col= 0, header = None)
y_test = pd.read_csv('../data/numerical/y_test.csv', sep = ';',index_col = 0, header = None)

In [4]:
#y_train = y_train.sample(2000, replace = False)
#y_test = y_test.sample(600, replace = False)

In [5]:
train_index = y_train.index.to_list()
test_index = y_test.index.to_list()

In [6]:
X_trainImg = loaded_images[train_index]
X_testImg = loaded_images[test_index]

In [7]:
y_train = y_train.values
y_test = y_test.values

maxPrice = y_train.max()
y_train = y_train / maxPrice
y_test = y_test / maxPrice

In [8]:
#np.save('../data/images/X_train.npy', X_trainImg)
#np.save('../data/images/X_test.npy', X_testImg)

In [9]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
    # initialize the input shape and channel dimension, assuming
    # TensorFlow/channels-last ordering
    inputShape = (height, width, depth)
    chanDim = -1

    # define the model input
    inputs = Input(shape=inputShape)
    # loop over the number of filters
    for (i, f) in enumerate(filters):
        # if this is the first CONV layer then set the input appropriately
        if i == 0:
            x = inputs
        # CONV => RELU => BN => POOL
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Activation("relu")(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

    # flatten the volume, then FC => RELU => BN => DROPOUT
    x = Flatten()(x)
    x = Dense(16)(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    # apply another FC layer, this one to match the number of nodes coming out of the MLP
    x = Dense(4)(x)
    x = Activation("relu")(x)
    # check to see if the regression node should be added
    x = Dense(1, activation="linear")(x)
    # construct the CNN
    model = Model(inputs, x)
    # return the CNN
    return model

In [10]:
model = create_cnn(224, 224, 3)

opt = Adam(lr=0.001, decay=1e-3/ 200)
'''model.compile(
    optimizer=opt, 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)'''
model.compile(loss="mean_absolute_error", optimizer=opt)

# train the model
print("[INFO] training model...")
# simple early stopping
es = EarlyStopping(monitor='val_loss', verbose=1)

model.fit(
    X_trainImg, 
    y_train, 
    epochs=100, 
    validation_data=(X_testImg, y_test), 
    batch_size=16,
    callbacks=[es]
)


[INFO] training model...
Epoch 1/100
125/125 [==============================] - 62s 494ms/step - loss: 0.4284 - val_loss: 0.1794
Epoch 2/100
125/125 [==============================] - 56s 449ms/step - loss: 0.1823 - val_loss: 0.1461
Epoch 3/100
125/125 [==============================] - 56s 447ms/step - loss: 0.1438 - val_loss: 0.1415
Epoch 4/100
125/125 [==============================] - 57s 454ms/step - loss: 0.1342 - val_loss: 0.1276
Epoch 5/100
125/125 [==============================] - 55s 438ms/step - loss: 0.1326 - val_loss: 0.1249
Epoch 6/100
125/125 [==============================] - 57s 452ms/step - loss: 0.1320 - val_loss: 0.1345
Epoch 00006: early stopping


In [11]:
model.save('CNN_model_reg.h5') 